# Feature Engineering

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')
import json

## Helper to reduce memory of dataframes

In [2]:
# function to reduce memory usage of pandas dataframes by casting columns to the most memory efficient data type

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## 1) Lag features

This function generates lag features. The function takes in NUM_LAG_DAYS as a parameter, which specifies how many days into the past we are looking at. 

Pre-condition: NUM_LAG_DAYS must be a multiple of 7

In [3]:
# function to obtain lag features

def getLagFeatures(features, NUM_LAG_DAYS):
    for i in range(1,NUM_LAG_DAYS+1):
        curr = 'lag'+str(i)
        features[curr] = features.groupby('id')['sales'].shift(i)
    features = features.dropna()
    return features

## 2) Rolling aggregates

This function generates rolling window features. The window is defined to be 7 days, so we generate the maximum, minimum, mean, sum and standard deviation for k 7-day periods where k = number of weeks into the past we are looking at.

In [4]:
# function to obtain rolling window features (mean, sum, max, min, std)

def getAggregates(features, NUM_LAG_DAYS):
    num_weeks = NUM_LAG_DAYS // 7
    for i in range(1,num_weeks+1):
        start = (i-1)*7+1
        end = start + 7
        cols = ['lag'+str(j) for j in range(start,end)]
        same_day_cols = ['lag'+str(k*7) for k in range(1,num_weeks+1)]
        features['lag_week'+str(i)+'_sum'] = features[cols].sum(axis=1)
        features['lag_week'+str(i)+'_mean'] = features[cols].mean(axis=1)
        features['lag_week'+str(i)+'_min'] = features[cols].min(axis=1)
        features['lag_week'+str(i)+'_max'] = features[cols].max(axis=1)
        features['lag_week'+str(i)+'_std'] = features[cols].std(axis=1)
        features['same_day_mean'] = features[same_day_cols].mean(axis=1)
        features['same_day_std'] = features[same_day_cols].std(axis=1)
        if i == 1:
            continue
        else:
            features['lag_week_diff'+str(i-1)] = features['lag_week'+str(i)+'_mean']-features['lag_week'+str(i-1)+'_mean']
    return features

## Price features

From the price dataset, we can generate several features related to the product price, such as maximum/minimum sell price and standard deviation.

In [5]:
# function to obtain price features

def getPriceFeatures(prices,id_map):
    prices['last_week_sell_price'] = prices.groupby(['store_id','item_id'])['sell_price'].shift(1)
    prices['max_sell_price'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('max')
    prices['min_sell_price'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('min')
    prices['mean_sell_price'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('mean')
    prices['price_diff'] = prices['sell_price'] - prices['last_week_sell_price']
    prices['price_std'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('std')
    prices['price_norm'] = prices['sell_price']/prices['max_sell_price']
    prices['price_unique'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
    prices['item_unique'] = prices.groupby(['store_id','sell_price'])['item_id'].transform('nunique')
    prices = id_map.merge(prices, on=['store_id','item_id'],how='left')
    return prices

## Date features

For date features, they are mostly given in the calendar dataset. The only preprocessing step to be done is to convert some of the columns to category type so as to further reduce memory usage. 

In [6]:
# function to obtain date features

def getDateFeatures(calendar):
    calendar = calendar.fillna('None')
    calendar['is_weekend'] = calendar['wday'].apply(lambda x: 1 if (x==1 or x==2) else 0)
    calendar['is_weekend'] = calendar['is_weekend'].astype('category')
    calendar['event_type_1'] = calendar['event_type_1'].astype('category')
    calendar['event_name_1'] = calendar['event_name_1'].astype('category')
    calendar['event_type_2'] = calendar['event_type_2'].astype('category')
    calendar['event_name_2'] = calendar['event_name_2'].astype('category')
    calendar['snap_CA'] = calendar['snap_CA'].astype('uint8')
    calendar['snap_TX'] = calendar['snap_TX'].astype('uint8')
    calendar['snap_WI'] = calendar['snap_WI'].astype('uint8')
    return calendar[['d','wm_yr_wk','wday','month','event_name_1','event_name_2','event_type_1','event_type_2','snap_CA','snap_TX','snap_WI']]


In [7]:
# helper function to set up a base dataframe for merging purposes

def getBaseFeatures(ids, days, calendar, prices):
    base_days = np.asarray([[d]*len(ids) for d in days]).flatten()
    base_ids = ids*len(days)
    base = pd.DataFrame({'id': base_ids,'d':base_days})
    base = base.merge(calendar,on=['d'],how='left')
    base = base.merge(prices, on=['wm_yr_wk','id'],how='left')
    return base

## Read in datasets

In [9]:
# initialize sales
sales = pd.read_pickle('sales_grid.pkl')
sales = sales[sales.d >= 1069] # 2014 onwards

# intialize calendar
calendar = pd.read_csv('calendar.csv')
calendar.d = calendar.d.apply(lambda d: int(d.split('_')[1])).apply(pd.to_numeric,downcast='unsigned')
calendar = calendar[calendar.d >= 1069] #2014 onwards
calendar = reduce_mem_usage(calendar)

# initialize prices
prices = pd.read_csv('sell_prices.csv')
prices = reduce_mem_usage(prices)

# initialize id map
id_map = pd.read_csv('id_map.csv')
stores = sales.store_id.unique()

Mem. usage decreased to  0.06 Mb (44.2% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


## Generate features at store level

We generate and save features at store level to work within the kernel memory li

In [11]:
NUM_LAG_DAYS = 28

for store in stores:
    store_features = sales[sales.store_id == store]
    store_features = getLagFeatures(store_features,NUM_LAG_DAYS)
    store_features = getAggregates(store_features,NUM_LAG_DAYS)
    store_features = store_features.drop(['item_id','store_id'],axis=1)
    ids = list(store_features.id.unique())
    days = list(calendar.d.values)
    base = getBaseFeatures(ids,days,getDateFeatures(calendar),getPriceFeatures(prices,id_map))
    store_features = base.merge(store_features,on=['id','d'],how='left')
    store_features = reduce_mem_usage(store_features)
    store_features.to_pickle(store+"_features.pkl")

Mem. usage decreased to 458.48 Mb (65.8% reduction)
Mem. usage decreased to 458.48 Mb (65.8% reduction)
Mem. usage decreased to 458.48 Mb (65.8% reduction)
Mem. usage decreased to 458.48 Mb (65.8% reduction)
Mem. usage decreased to 458.48 Mb (65.8% reduction)
Mem. usage decreased to 458.48 Mb (65.8% reduction)
Mem. usage decreased to 458.48 Mb (65.8% reduction)
Mem. usage decreased to 458.48 Mb (65.8% reduction)
Mem. usage decreased to 458.48 Mb (65.8% reduction)
Mem. usage decreased to 458.48 Mb (65.8% reduction)
